In [38]:
#Importation des packages

import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import os 

In [39]:
# Definir le repertoire de travail 
pwd = "C:\\Users\\mouss\\OneDrive\\Documents\\TotalEnergies" 
os.chdir(pwd) 
 
# Afficher le nouveau repertoire de travail 
print(os.getcwd()) 

C:\Users\mouss\OneDrive\Documents\TotalEnergies


In [43]:
tran = pd.read_csv("./data/table_transactions.csv",sep = ";"
                       #,index_col = "EP_CODE"
                      , encoding= 'latin-1')

In [67]:
tran.head()

,EP_CODE,TR_PRODUCT_CODE,TR_PRODUCT_TYPE,TR_SUPPORT_CODE,TR_DATE,TR_REVENUE,TR_VOLUME,TR_COUNT
0,87322382,730,Non-Fuel,C100,2021-08-02,132.12,0.00,35
1,87297430,102,Fuel,C100,2021-08-03,132.54,102.68,4
2,87128600,222,Fuel,C200,2021-08-03,323.46,227.94,11
3,87240226,112,Fuel,C700,2021-08-03,75.84,57.27,1
4,87185411,102,Fuel,C100,2021-08-03,2551.19,2036.94,41


In [68]:
tran.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1314498 entries, 0 to 1314497
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   EP_CODE          1314498 non-null  int64  
 1   TR_PRODUCT_CODE  1314498 non-null  int64  
 2   TR_PRODUCT_TYPE  1314498 non-null  object 
 3   TR_SUPPORT_CODE  1312833 non-null  object 
 4   TR_DATE          1314498 non-null  object 
 5   TR_REVENUE       1314498 non-null  float64
 6   TR_VOLUME        1312477 non-null  float64
 7   TR_COUNT         1314498 non-null  int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 80.2+ MB


In [69]:
tran['TR_DATE'] = pd.to_datetime(tran['TR_DATE'])
tran.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1314498 entries, 0 to 1314497
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   EP_CODE          1314498 non-null  int64         
 1   TR_PRODUCT_CODE  1314498 non-null  int64         
 2   TR_PRODUCT_TYPE  1314498 non-null  object        
 3   TR_SUPPORT_CODE  1312833 non-null  object        
 4   TR_DATE          1314498 non-null  datetime64[ns]
 5   TR_REVENUE       1314498 non-null  float64       
 6   TR_VOLUME        1312477 non-null  float64       
 7   TR_COUNT         1314498 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 80.2+ MB


# RFM

In [71]:
df_rfm = tran[['EP_CODE','TR_DATE','TR_REVENUE','TR_COUNT']]

In [72]:
df_rfm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1314498 entries, 0 to 1314497
Data columns (total 4 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   EP_CODE     1314498 non-null  int64         
 1   TR_DATE     1314498 non-null  datetime64[ns]
 2   TR_REVENUE  1314498 non-null  float64       
 3   TR_COUNT    1314498 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 40.1 MB


In [73]:
PRESENT = dt.datetime(2021,8,14)  # Definir le present comme le 14 août 2021
PRESENT

datetime.datetime(2021, 8, 14, 0, 0)

In [74]:
rfm = df_rfm.groupby('EP_CODE').agg({'TR_DATE': lambda date: (PRESENT - date.max()).days,
                                        'TR_COUNT': lambda num: num.sum(),
                                        'TR_REVENUE': lambda revenue: revenue.sum()})

In [75]:
rfm.head()

,TR_DATE,TR_COUNT,TR_REVENUE
EP_CODE,,,
87000019,3,67,327.35
87000052,2,15,523.47
87000070,1,55,3500.22
87000076,2,31,471.55
87000088,3,3,189.35


In [76]:
rfm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 138527 entries, 87000019 to 87989974
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   TR_DATE     138527 non-null  int64  
 1   TR_COUNT    138527 non-null  int64  
 2   TR_REVENUE  138527 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 4.2 MB


In [77]:
# Changer le nom des colonnes 

rfm.columns=['recency','frequency','monetary']

In [78]:
rfm.head()

,recency,frequency,monetary
EP_CODE,,,
87000019,3,67,327.35
87000052,2,15,523.47
87000070,1,55,3500.22
87000076,2,31,471.55
87000088,3,3,189.35


In [79]:
rfm.describe()

,recency,frequency,monetary
count,138527.000000,138527.000000,1.385270e+05
mean,3.562129,32.409855,1.126568e+03
std,3.104698,260.024939,1.659381e+04
min,1.000000,1.000000,1.300000e-01
25%,1.000000,2.000000,8.463500e+01
50%,2.000000,6.000000,2.073000e+02
75%,5.000000,18.000000,6.103750e+02
max,12.000000,49945.000000,5.243186e+06


# Récence
On remarque que plus du tiers des clients ont effectués une transaction très recemment, c'est à dire à la date du **13 août 2021**. Donc on ne pourra faire que deux groupes à partir de la variable **récence**. 

In [80]:
rfm['recency'].value_counts(normalize=True)

1     0.376150
2     0.156525
3     0.110419
4     0.072094
5     0.056126
8     0.053758
7     0.036275
9     0.035141
6     0.031741
10    0.028312
11    0.022653
12    0.020805
Name: recency, dtype: float64

# Frequence

In [81]:
rfm['frequency'].value_counts(normalize=True)

1       0.170248
2       0.120525
3       0.084265
4       0.064522
5       0.050315
          ...   
1484    0.000007
1868    0.000007
794     0.000007
2075    0.000007
1919    0.000007
Name: frequency, Length: 1223, dtype: float64

# Monétaire

In [82]:
rfm['monetary'].value_counts(normalize=True)

41.67      0.001198
50.00      0.000881
3.00       0.000642
66.67      0.000606
25.00      0.000578
             ...   
428.67     0.000007
486.65     0.000007
3012.12    0.000007
500.99     0.000007
184.32     0.000007
Name: monetary, Length: 85000, dtype: float64

# Segmentation

In [83]:
rfm['r_median'] = pd.qcut(rfm['recency'], 2, ['2','1'])
rfm['f_quintile'] = pd.qcut(rfm['frequency'], 5, ['1','2','3','4','5'])
rfm['m_quintile'] = pd.qcut(rfm['monetary'], 5, ['1','2','3','4','5'])

In [84]:
rfm.head()

,recency,frequency,monetary,r_median,f_quintile,m_quintile
EP_CODE,,,,,,
87000019,3,67,327.35,1,5,4
87000052,2,15,523.47,2,4,4
87000070,1,55,3500.22,2,5,5
87000076,2,31,471.55,2,5,4
87000088,3,3,189.35,1,2,3


# Score des clients

In [85]:
rfm['RFM_Score'] = rfm.r_median.astype(str)+ rfm.f_quintile.astype(str) + rfm.m_quintile.astype(str)
rfm.head()

,recency,frequency,monetary,r_median,f_quintile,m_quintile,RFM_Score
EP_CODE,,,,,,,
87000019,3,67,327.35,1,5,4,154
87000052,2,15,523.47,2,4,4,244
87000070,1,55,3500.22,2,5,5,255
87000076,2,31,471.55,2,5,4,254
87000088,3,3,189.35,1,2,3,123


In [86]:
rfm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 138527 entries, 87000019 to 87989974
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   recency     138527 non-null  int64   
 1   frequency   138527 non-null  int64   
 2   monetary    138527 non-null  float64 
 3   r_median    138527 non-null  category
 4   f_quintile  138527 non-null  category
 5   m_quintile  138527 non-null  category
 6   RFM_Score   138527 non-null  object  
dtypes: category(3), float64(1), int64(2), object(1)
memory usage: 5.7+ MB


# Meilleurs clients

In [89]:
top_customers = rfm[rfm['RFM_Score'].isin(['155'])].sort_values('monetary', ascending=False)
top_customers.head()

,recency,frequency,monetary,r_median,f_quintile,m_quintile,RFM_Score
EP_CODE,,,,,,,
87052439,8,182,20771.99,1,5,5,155
87206917,4,340,12771.07,1,5,5,155
87364004,4,103,11474.24,1,5,5,155
87127525,5,290,8537.68,1,5,5,155
87263311,3,31,8148.99,1,5,5,155


In [90]:
top_customers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 464 entries, 87052439 to 87060674
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   recency     464 non-null    int64   
 1   frequency   464 non-null    int64   
 2   monetary    464 non-null    float64 
 3   r_median    464 non-null    category
 4   f_quintile  464 non-null    category
 5   m_quintile  464 non-null    category
 6   RFM_Score   464 non-null    object  
dtypes: category(3), float64(1), int64(2), object(1)
memory usage: 20.0+ KB


# Clients à risque

In [91]:
risky_customers = rfm[rfm['RFM_Score']=='111'].sort_values('monetary', ascending=False)
risky_customers.head()

,recency,frequency,monetary,r_median,f_quintile,m_quintile,RFM_Score
EP_CODE,,,,,,,
87254500,9,1,69.36,1,1,1,111
87288320,7,1,69.35,1,1,1,111
87761786,3,1,69.35,1,1,1,111
87361276,6,1,69.34,1,1,1,111
87962431,11,1,69.34,1,1,1,111


In [92]:
risky_customers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18487 entries, 87254500 to 87147572
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   recency     18487 non-null  int64   
 1   frequency   18487 non-null  int64   
 2   monetary    18487 non-null  float64 
 3   r_median    18487 non-null  category
 4   f_quintile  18487 non-null  category
 5   m_quintile  18487 non-null  category
 6   RFM_Score   18487 non-null  object  
dtypes: category(3), float64(1), int64(2), object(1)
memory usage: 776.8+ KB


In [95]:
rfm.to_csv('./data/rfm.csv',sep=';',encoding='latin-1')

In [94]:
help(rfm.to_csv)

Help on method to_csv in module pandas.core.generic:

to_csv(path_or_buf: 'Optional[FilePathOrBuffer]' = None, sep: 'str' = ',', na_rep: 'str' = '', float_format: 'Optional[str]' = None, columns: 'Optional[Sequence[Label]]' = None, header: 'Union[bool_t, List[str]]' = True, index: 'bool_t' = True, index_label: 'Optional[IndexLabel]' = None, mode: 'str' = 'w', encoding: 'Optional[str]' = None, compression: 'CompressionOptions' = 'infer', quoting: 'Optional[int]' = None, quotechar: 'str' = '"', line_terminator: 'Optional[str]' = None, chunksize: 'Optional[int]' = None, date_format: 'Optional[str]' = None, doublequote: 'bool_t' = True, escapechar: 'Optional[str]' = None, decimal: 'str' = '.', errors: 'str' = 'strict', storage_options: 'StorageOptions' = None) -> 'Optional[str]' method of pandas.core.frame.DataFrame instance
    Write object to a comma-separated values (csv) file.
    
    .. versionchanged:: 0.24.0
        The order of arguments for Series was changed.
    
    Parameters